<a href="https://colab.research.google.com/github/MohamedHesham02/Sentiment-Analysis-Arabic-Tweets/blob/main/sentiment_analysis_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, LSTM,Embedding
from tensorflow.keras.layers import GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D ,Dropout
from tensorflow.keras.optimizers import SGD

# Tri-Grams

In [ ]:
trigrams_features = pd.read_csv('trigrams_features.csv')  
labels = pd.read_csv('labels.csv')

In [ ]:
del trigrams_features['Unnamed: 0']
del labels['Unnamed: 0']

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(trigrams_features, labels,test_size=0.2)
print('Training Data:', X_train.shape[0])
print('Test Data:', X_test.shape[0])

Training Data: 16666
Test Data: 4167


In [ ]:
X_train.shape

(16666, 10000)

In [ ]:
trigram_model = Sequential()
trigram_model.add(Dense(128, input_shape=((trigrams_features.shape)[1],), activation='tanh'))
trigram_model.add(Dropout(0.5))
trigram_model.add(Dense(64, activation='tanh'))
trigram_model.add(Dropout(0.5))
trigram_model.add(Dense(18, activation='softmax'))

sgd = SGD(learning_rate=0.001)
trigram_model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = trigram_model.fit(X_train, y_train, epochs=40, batch_size=32, verbose=1, validation_split=0.2)

Epoch 1/40
417/417 [==============================] - 7s 15ms/step - loss: 2.8786 - accuracy: 0.0845 - val_loss: 2.8689 - val_accuracy: 0.0852
Epoch 2/40
417/417 [==============================] - 6s 14ms/step - loss: 2.8665 - accuracy: 0.0857 - val_loss: 2.8632 - val_accuracy: 0.0870
Epoch 3/40
417/417 [==============================] - 6s 14ms/step - loss: 2.8633 - accuracy: 0.0872 - val_loss: 2.8605 - val_accuracy: 0.0897
Epoch 4/40
417/417 [==============================] - 6s 14ms/step - loss: 2.8631 - accuracy: 0.0872 - val_loss: 2.8587 - val_accuracy: 0.0912
Epoch 5/40
417/417 [==============================] - 6s 14ms/step - loss: 2.8592 - accuracy: 0.0909 - val_loss: 2.8566 - val_accuracy: 0.0918
Epoch 6/40
417/417 [==============================] - 6s 14ms/step - loss: 2.8567 - accuracy: 0.0923 - val_loss: 2.8552 - val_accuracy: 0.0957
Epoch 7/40
417/417 [==============================] - 6s 14ms/step - loss: 2.8541 - accuracy: 0.0947 - val_loss: 2.8539 - val_accuracy: 0.0975

In [ ]:
embedding_dim=100
max_features = 10000
output_shape = 32

modelrnn=Sequential()
modelrnn.add(layers.Embedding(input_dim=max_features, output_dim=embedding_dim, input_length=(X_train.shape)[1]))

modelrnn.add(layers.LSTM(output_shape)) 
modelrnn.add(layers.Dropout(0.5))

modelrnn.add(layers.Dense(18, activation='softmax'))
modelrnn.compile(loss='sparse_categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])
modelrnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 10000, 100)        1000000   
                                                                 
 lstm (LSTM)                 (None, 32)                17024     
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 18)                594       
                                                                 
Total params: 1,017,618
Trainable params: 1,017,618
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history=modelrnn.fit(X_train,y_train, epochs=4, batch_size=512, verbose=1, validation_split=0.2)

Epoch 1/4


In [ ]:
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.metrics import precision_score, recall_score

svm = LinearSVC(max_iter=300, multi_class='ovr')
svm = svm.fit(X_train, y_train)

y_pred = svm.predict(X_test)
print('Accuracy : ', svm.score(X_test, y_test))
print("Precision of all classes: ", precision_score(y_test, y_pred, average=None))
print("Recall of all classes: ", recall_score(y_test, y_pred, average=None))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy :  0.19126469882409408
Precision of all classes:  [0.09619816 0.72222222 0.58536585 0.77083333 0.57142857 0.66666667
 0.70967742 0.66666667 0.91150442 0.76470588 0.3        0.38235294
 0.64       0.53333333 0.9        0.59090909 0.46376812 0.67391304]
Recall of all classes:  [0.93036212 0.14130435 0.09876543 0.14979757 0.02867384 0.16113744
 0.19469027 0.092827   0.45175439 0.10441767 0.02739726 0.05579399
 0.14159292 0.06425703 0.06976744 0.05416667 0.12851406 0.12350598]


In [ ]:
LRC = LogisticRegression()
LRC = LRC.fit(X_train, y_train)

y_pred = LRC.predict(X_test)

print('Accuracy : ', LRC.score(X_test, y_test))
print("Precision of all classes: ", precision_score(y_test, y_pred, average=None))
print("Recall of all classes: ", recall_score(y_test, y_pred, average=None))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Accuracy :  0.18622510199184064
Precision of all classes:  [0.09578107 1.         0.62162162 0.81395349 0.5        0.66
 0.72881356 0.67741935 0.91262136 0.78125    0.31578947 0.40625
 0.6        0.5483871  1.         0.65       0.41975309 0.75      ]
Recall of all classes:  [0.93593315 0.11956522 0.09465021 0.1417004  0.02867384 0.1563981
 0.19026549 0.08860759 0.4122807  0.10040161 0.02739726 0.05579399
 0.13274336 0.06827309 0.03100775 0.05416667 0.13654618 0.11952191]


# Bi-Grams

In [ ]:
Bigrams_features = pd.read_csv('bigrams_features.csv')

In [ ]:
del Bigrams_features['Unnamed: 0']

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(Bigrams_features, labels,test_size=0.2)
print('Training Data:', X_train.shape[0])
print('Test Data:', X_test.shape[0])

Training Data: 16666
Test Data: 4167


In [ ]:
bigram_model = Sequential()
bigram_model.add(Dense(128, input_shape=((Bigrams_features.shape)[1],), activation='tanh'))
bigram_model.add(Dropout(0.5))
bigram_model.add(Dense(64, activation='tanh'))
bigram_model.add(Dropout(0.5))
bigram_model.add(Dense(18, activation='softmax'))

# Compile model. Stochastic gradient descent for backpropagation with Nesterov accelerated gradient gives good results for this model 
sgd = SGD(learning_rate=0.001)
bigram_model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = bigram_model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=1, validation_split=0.2)

Epoch 1/20
417/417 [==============================] - 9s 20ms/step - loss: 2.8886 - accuracy: 0.0669 - val_loss: 2.8853 - val_accuracy: 0.0789
Epoch 2/20
417/417 [==============================] - 6s 13ms/step - loss: 2.8836 - accuracy: 0.0749 - val_loss: 2.8809 - val_accuracy: 0.0843
Epoch 3/20
417/417 [==============================] - 5s 13ms/step - loss: 2.8809 - accuracy: 0.0734 - val_loss: 2.8774 - val_accuracy: 0.0924
Epoch 4/20
417/417 [==============================] - 6s 15ms/step - loss: 2.8780 - accuracy: 0.0806 - val_loss: 2.8743 - val_accuracy: 0.0927
Epoch 5/20
417/417 [==============================] - 9s 22ms/step - loss: 2.8757 - accuracy: 0.0798 - val_loss: 2.8718 - val_accuracy: 0.0933
Epoch 6/20
417/417 [==============================] - 8s 20ms/step - loss: 2.8729 - accuracy: 0.0829 - val_loss: 2.8697 - val_accuracy: 0.0939
Epoch 7/20
417/417 [==============================] - 5s 13ms/step - loss: 2.8705 - accuracy: 0.0828 - val_loss: 2.8678 - val_accuracy: 0.0939

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.metrics import precision_score, recall_score

svm = LinearSVC(max_iter=300, multi_class='ovr')
svm = svm.fit(X_train, y_train)

y_pred = svm.predict(X_test)
print('Accuracy : ', svm.score(X_test, y_test))
print("Precision of all classes: ", precision_score(y_test, y_pred, average=None))
print("Recall of all classes: ", recall_score(y_test, y_pred, average=None))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy :  0.27837772978161746
Precision of all classes:  [0.13527851 0.31746032 0.2866242  0.44102564 0.24848485 0.38586957
 0.35602094 0.41176471 0.7        0.48344371 0.17857143 0.20218579
 0.40697674 0.2189781  0.30882353 0.30769231 0.36507937 0.33108108]
Recall of all classes:  [0.5698324  0.22727273 0.17175573 0.32575758 0.17446809 0.31555556
 0.32692308 0.31390135 0.51953125 0.3173913  0.10245902 0.1504065
 0.28225806 0.12931034 0.175      0.20338983 0.28512397 0.196     ]


In [ ]:
LRC = LogisticRegression()
LRC = LRC.fit(X_train, y_train)

y_pred = LRC.predict(X_test)

print('Accuracy : ', LRC.score(X_test, y_test))
print("Precision of all classes: ", precision_score(y_test, y_pred, average=None))
print("Recall of all classes: ", recall_score(y_test, y_pred, average=None))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Accuracy :  0.29469642428605713
Precision of all classes:  [0.14088557 0.53333333 0.3537415  0.48167539 0.27564103 0.42011834
 0.40106952 0.47468354 0.82248521 0.48275862 0.20325203 0.23333333
 0.46794872 0.20547945 0.66666667 0.32692308 0.44512195 0.33802817]
Recall of all classes:  [0.68435754 0.09090909 0.19847328 0.34848485 0.18297872 0.31555556
 0.36057692 0.33632287 0.54296875 0.30434783 0.10245902 0.17073171
 0.29435484 0.12931034 0.13333333 0.21610169 0.30165289 0.192     ]


# Uni-Grams

In [ ]:
unigram = pd.read_csv('unigrams_features.csv')

In [ ]:
unigram.head()

,Unnamed: 0,ءبر,ءبررر,ءبرررر,ءبش,ءبشو,ءثر,ءخر,ءدب,ءسس,...,ييز,ييس,ييش,ييض,ييغ,ييك,ييم,يين,ييه,ييي
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
del unigram['Unnamed: 0']

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(unigram, labels,test_size=0.2)
print('Training Data:', X_train.shape[0])
print('Test Data:', X_test.shape[0])

Training Data: 16666
Test Data: 4167


In [ ]:
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.metrics import precision_score, recall_score

svm = LinearSVC(max_iter=300, multi_class='ovr')
svm = svm.fit(X_train, y_train)

y_pred = svm.predict(X_test)
print('Accuracy : ', svm.score(X_test, y_test))
print("Precision of all classes: ", precision_score(y_test, y_pred, average=None))
print("Recall of all classes: ", recall_score(y_test, y_pred, average=None))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy :  0.4761219102471802
Precision of all classes:  [0.40509915 0.29069767 0.43612335 0.59591837 0.49173554 0.51291513
 0.50678733 0.53469388 0.70484581 0.64878049 0.31128405 0.35135135
 0.57261411 0.25688073 0.5        0.46017699 0.56273764 0.3744856 ]
Recall of all classes:  [0.40056022 0.33783784 0.41949153 0.58634538 0.52654867 0.57916667
 0.43921569 0.58482143 0.68376068 0.62149533 0.29739777 0.39224138
 0.52873563 0.24669604 0.52272727 0.42622951 0.6090535  0.364     ]


In [ ]:
LRC = LogisticRegression()
LRC = LRC.fit(X_train, y_train)

y_pred = LRC.predict(X_test)

print('Accuracy : ', LRC.score(X_test, y_test))
print("Precision of all classes: ", precision_score(y_test, y_pred, average=None))
print("Recall of all classes: ", recall_score(y_test, y_pred, average=None))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Accuracy :  0.5121190304775618
Precision of all classes:  [0.40807175 0.47916667 0.42168675 0.63059701 0.51171875 0.57874016
 0.53043478 0.53036437 0.77619048 0.6713615  0.36051502 0.39272727
 0.62337662 0.28899083 0.62857143 0.48387097 0.63114754 0.42152466]
Recall of all classes:  [0.50980392 0.31081081 0.44491525 0.67871486 0.57964602 0.6125
 0.47843137 0.58482143 0.6965812  0.6682243  0.31226766 0.46551724
 0.55172414 0.27753304 0.5        0.43032787 0.63374486 0.376     ]


# Highest Accuracy Achieved at Unigrams in Machine Learning using Logistic Regression 
## So, I saved it

In [ ]:
!pip install joblib
import joblib
filename = 'finalized_model.sav'
joblib.dump(LRC, filename)

['finalized_model.sav']

In [ ]:
unigram_model = Sequential()
unigram_model.add(Dense(128, input_shape=((unigram.shape)[1],), activation='tanh'))
unigram_model.add(Dropout(0.5))
unigram_model.add(Dense(64, activation='tanh'))
unigram_model.add(Dropout(0.5))
unigram_model.add(Dense(18, activation='softmax'))

sgd = SGD(learning_rate=0.001)
unigram_model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = unigram_model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1, validation_split=0.2)

Epoch 1/100
417/417 [==============================] - 13s 29ms/step - loss: 2.8940 - accuracy: 0.0597 - val_loss: 2.8815 - val_accuracy: 0.0756
Epoch 2/100
417/417 [==============================] - 9s 22ms/step - loss: 2.8824 - accuracy: 0.0708 - val_loss: 2.8724 - val_accuracy: 0.0954
Epoch 3/100
417/417 [==============================] - 7s 16ms/step - loss: 2.8733 - accuracy: 0.0804 - val_loss: 2.8645 - val_accuracy: 0.0978
Epoch 4/100
417/417 [==============================] - 6s 13ms/step - loss: 2.8649 - accuracy: 0.0896 - val_loss: 2.8573 - val_accuracy: 0.1023
Epoch 5/100
417/417 [==============================] - 6s 13ms/step - loss: 2.8575 - accuracy: 0.0875 - val_loss: 2.8506 - val_accuracy: 0.1035
Epoch 6/100
417/417 [==============================] - 6s 13ms/step - loss: 2.8501 - accuracy: 0.0941 - val_loss: 2.8443 - val_accuracy: 0.1071
Epoch 7/100
417/417 [==============================] - 5s 13ms/step - loss: 2.8438 - accuracy: 0.0992 - val_loss: 2.8382 - val_accuracy

# Also obviously, Highest Accuracy in Deep Learning Achieved at Unigrams using Fully Connected Layers 

In [ ]:
unigram_model.save('pre-trained.h5')

# Sequence Text

In [ ]:
seq_data = pd.read_csv('seq_data.csv')

In [ ]:
del seq_data['Unnamed: 0']

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(seq_data, labels,test_size=0.2)
print('Training Data:', X_train.shape[0])
print('Test Data:', X_test.shape[0])

Training Data: 16666
Test Data: 4167


In [ ]:
seqdata_model = Sequential()
seqdata_model.add(Dense(128, input_shape=((seq_data.shape)[1],), activation='tanh'))
seqdata_model.add(Dropout(0.5))
seqdata_model.add(Dense(64, activation='tanh'))
seqdata_model.add(Dropout(0.5))
seqdata_model.add(Dense(18, activation='softmax'))

sgd = SGD(learning_rate=0.001)
seqdata_model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = seqdata_model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=1, validation_split=0.2)

Epoch 1/20
417/417 [==============================] - 2s 3ms/step - loss: 3.6158 - accuracy: 0.0533 - val_loss: 3.0761 - val_accuracy: 0.0663
Epoch 2/20
417/417 [==============================] - 1s 2ms/step - loss: 3.5344 - accuracy: 0.0626 - val_loss: 3.0446 - val_accuracy: 0.0672
Epoch 3/20
417/417 [==============================] - 1s 2ms/step - loss: 3.4852 - accuracy: 0.0622 - val_loss: 3.0304 - val_accuracy: 0.0612
Epoch 4/20
417/417 [==============================] - 1s 2ms/step - loss: 3.4491 - accuracy: 0.0642 - val_loss: 2.9981 - val_accuracy: 0.0615
Epoch 5/20
417/417 [==============================] - 1s 2ms/step - loss: 3.4232 - accuracy: 0.0645 - val_loss: 2.9922 - val_accuracy: 0.0696
Epoch 6/20
417/417 [==============================] - 1s 2ms/step - loss: 3.3794 - accuracy: 0.0613 - val_loss: 2.9868 - val_accuracy: 0.0705
Epoch 7/20
417/417 [==============================] - 1s 2ms/step - loss: 3.3610 - accuracy: 0.0632 - val_loss: 2.9746 - val_accuracy: 0.0690
Epoch 

In [ ]:
embedding_dim=100

model=Sequential()
model.add(layers.Embedding(input_dim=max_features, output_dim=embedding_dim, input_length=(X_train.shape)[1]))

model.add(layers.LSTM(64))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(18, activation='relu'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])
model.summary()

In [ ]:
history=seqdata_model.fit(X_train,y_train, epochs=20, batch_size=128, verbose=1, validation_split=0.2)

Epoch 1/20
45/45 [==============================] - 16s 233ms/step - loss: 6.0339 - accuracy: 0.0516 - val_loss: 5.2582 - val_accuracy: 0.0597
Epoch 2/20
45/45 [==============================] - 10s 225ms/step - loss: 5.4072 - accuracy: 0.0618 - val_loss: 5.1338 - val_accuracy: 0.0569
Epoch 3/20
45/45 [==============================] - 10s 218ms/step - loss: 5.5890 - accuracy: 0.0700 - val_loss: 5.3575 - val_accuracy: 0.0535
Epoch 4/20
45/45 [==============================] - 10s 217ms/step - loss: 5.5495 - accuracy: 0.0580 - val_loss: 5.8217 - val_accuracy: 0.0590
Epoch 5/20
45/45 [==============================] - 10s 218ms/step - loss: 5.6451 - accuracy: 0.0611 - val_loss: 5.4243 - val_accuracy: 0.0486
Epoch 6/20
45/45 [==============================] - 10s 219ms/step - loss: 5.6216 - accuracy: 0.0576 - val_loss: 5.3177 - val_accuracy: 0.0486
Epoch 7/20
45/45 [==============================] - 10s 218ms/step - loss: 5.3668 - accuracy: 0.0762 - val_loss: 5.7307 - val_accuracy: 0.0542

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.metrics import precision_score, recall_score

svm = LinearSVC(max_iter=300, multi_class='ovr')
svm = svm.fit(X_train, y_train)

y_pred = svm.predict(X_test)
print('Accuracy : ', svm.score(X_test, y_test))
print("Precision of all classes: ", precision_score(y_test, y_pred, average=None))
print("Recall of all classes: ", recall_score(y_test, y_pred, average=None))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy :  0.0650347972162227
Precision of all classes:  [0.08152174 0.         0.05952381 0.06666667 0.         0.03921569
 0.04518072 0.02631579 0.16666667 0.         0.1        0.
 0.05555556 0.06741573 0.         0.066233   0.08510638 0.06038647]
Recall of all classes:  [0.11749347 0.         0.04032258 0.04621849 0.         0.0456621
 0.06355932 0.01777778 0.075      0.         0.00420168 0.
 0.01702128 0.05063291 0.         0.4534413  0.0167364  0.09578544]


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


In [ ]:
LRC = LogisticRegression()
LRC = LRC.fit(X_train, y_train)

y_pred = LRC.predict(X_test)

print('Accuracy : ', LRC.score(X_test, y_test))
print("Precision of all classes: ", precision_score(y_test, y_pred, average=None))
print("Recall of all classes: ", recall_score(y_test, y_pred, average=None))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy :  0.08999280057595392
Precision of all classes:  [0.09059745 0.         0.10714286 0.04081633 0.         0.13829787
 0.12676056 0.2        0.15777262 0.0506156  0.07142857 0.03448276
 0.05789474 0.         0.         0.07352941 0.06930693 0.05825243]
Recall of all classes:  [0.48302872 0.         0.01209677 0.00840336 0.         0.17808219
 0.03813559 0.00444444 0.28333333 0.17619048 0.00420168 0.00390625
 0.04680851 0.         0.         0.02024291 0.0292887  0.02298851]


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


# Findings

### The scores are too low compared to models that are usable in real world due to the slow fetching of tweets as decribed in the presentation, but this is just a demo of Arabic language preprocessing and modelling.

### The highest accuracies are achieved at unigrams whether in machine or deep learning while it should not be and the best should be in trigrams, but after thinking about the reason, I thought about that is due to compound verbs (e.g. Carry out, catch up, look for, ...etc.) that is occurring too much in English while it's not occuring at the same familiarity in Arabic language so, the best N-Gram model in Arabic text processing is Uni-Grams 

### Proof: To make sure of the efficiency of the model that can achieve high scores if there's a big dataset to be trained on the same preprocesing and modelling technique and changing some hyperparamters to be compatible with modelling of this dataset, This approach is found in proof_of_efficieny.ipynb , The scores in both Machine Learning Algorithms ~= 91 % and in Deep Learning Fully Connected Network, Accuracy is ~= 93 %  


### Finally, We saved the highest scores models in machine and deep learning to be deployed and tested by FAST API 